### Transfer Learning Bölümü

In [ ]:
# Kaynak: https://www.kaggle.com/code/mahmoudftolba/transfer-learning-clearly-explained/notebook
# Linkteki modeli kullanacağız

In [1]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet' , include_top = False , 
                  input_shape=(224,224,3))
# Resizing the input data

In [2]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# We only add a new classifier (Trainable params: 1,605,954) , which will be trained from scratch, 
# on top of the pretrained model so that we can repurpose the previously learned feature maps for our dataset by training the model 
# for a few epochs till the top layers are pretty well trained

In [3]:
for layer in base_model.layers:
    layer.trainable = False

In [4]:
from tensorflow import keras
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output
x = keras.layers.Flatten()(last_output)
x = keras.layers.Dense(64, activation='relu', name='FC_1')(x)
# BatchNormalization
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(2, activation='sigmoid', name='sigmoid')(x)

In [5]:
VGG16_model = keras.Model(inputs = base_model.input, outputs = x)

In [6]:
VGG16_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
VGG16_model.compile(optimizer=keras.optimizers.Adam(),loss = 'binary_crossentropy', metrics=["accuracy"])

In [8]:
train_path = './data/train'
valid_path = './data/valid'

train_batches = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(train_path,
                        target_size=(224,224),
                        batch_size=10)
valid_batches = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(valid_path,
                        target_size=(224,224),
                        batch_size=30)

Found 19952 images belonging to 2 classes.
Found 4988 images belonging to 2 classes.


In [9]:
VGG16_model.fit(train_batches, validation_data = valid_batches,
                           epochs = 10 , steps_per_epoch=10  )

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 0.7963 - accuracy: 0.6900

c:\Users\mcsahin\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


10/10 [==============================] - 255s 28s/step - loss: 0.7963 - accuracy: 0.6900 - val_loss: 1.0342 - val_accuracy: 0.8392
Epoch 2/10
10/10 [==============================] - 255s 28s/step - loss: 0.4451 - accuracy: 0.8900 - val_loss: 0.3554 - val_accuracy: 0.9148
Epoch 3/10
10/10 [==============================] - 261s 29s/step - loss: 0.3657 - accuracy: 0.8800 - val_loss: 0.2352 - val_accuracy: 0.9274
Epoch 4/10
10/10 [==============================] - 257s 29s/step - loss: 0.4474 - accuracy: 0.8600 - val_loss: 0.1825 - val_accuracy: 0.9401
Epoch 5/10
10/10 [==============================] - 260s 29s/step - loss: 0.3456 - accuracy: 0.8600 - val_loss: 0.1556 - val_accuracy: 0.9449
Epoch 6/10
10/10 [==============================] - 261s 29s/step - loss: 0.3734 - accuracy: 0.8800 - val_loss: 0.1562 - val_accuracy: 0.9461
Epoch 7/10
10/10 [==============================] - 261s 29s/step - loss: 0.3572 - accuracy: 0.8900 - val_loss: 0.1561 - val_accuracy: 0.9513
Epoch 8/10
10/10 

In [ ]:
# We reach %96 accuracy!